# Euclidean Alignment + Weighted TL

## Load dataset

In [ ]:
import sys

In [ ]:
sys.path.append('D:\Google_Drive\JupyterNotebookProjects\bci-research\plan_c\csp')
sys.path.append('D:\Google_Drive\JupyterNotebookProjects\bci-research\plan_c\func')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import moabb.datasets
import moabb.paradigms


from copy import deepcopy
from csp.utils import subject_counter
from csp.preprocess import fir_bandpass, apply_bandpass, fetch_left_right_EEG
from csp.preprocess import split_EEG_one_class, process_s_data
from csp.feat_extraction import compute_Z, feat_vector, true_label
from csp.csp import CSP

In [4]:
dframe = pd.DataFrame

In [5]:
# First import the data

# Number of subject, n + 1 for iteration purpose (there are 9 subjects)
ns = 10

# Creating dict to store original data and modified data
# ori_data will serve as initial loaded data that will remain unchanged
ori_data = dict()
mod_data = dict() 

## Load Dataset

In [6]:
# Iter over all data path then store them in sub0X variable
for subj in range(1, 10):
    data_path = '../../datasets/BCICIV2a/A{:02d}T.npz'.format(subj)
    
    # Load EEG data from datapath and store into subj0X variabel then store into ori_dict
    # Then also fetch 's' (EEG data) into mod_data
    ori_data[subj] = np.load(data_path)
    mod_data[subj] = {}
    mod_data[subj]['s'] = deepcopy(ori_data[subj]['s'])
    mod_data[subj]['etyp'] = deepcopy(ori_data[subj]['etyp'])
    mod_data[subj]['epos'] = deepcopy(ori_data[subj]['epos'])
    
    # Remove last three EOG electrodes
    mod_data[subj]['s'] = np.delete(mod_data[subj]['s'], np.s_[22:], 1)

## Preprocessing

### Apply Bandpass Filter

In [7]:
from scipy.signal import firwin, freqs, lfilter

In [8]:
# Frequency sampling at 250 Hz
fs = 250

# Applying bandpass filter 8-30 Hz to all subjects raw EEG
b = fir_bandpass(51, low=8, high=30, fs=fs)

In [9]:
## PREPROCESSING ##

# Frequency sampling at 250 Hz
fs = 250

# Applying bandpass filter 8-30 Hz to all subjects raw EEG
b = fir_bandpass(51, low=8, high=30, fs=fs)

for subj in mod_data.keys():
    temp_raw_EEG = mod_data[subj]['s']
    
    mod_data[subj]['EEG_filtered'] = apply_bandpass(temp_raw_EEG, b)

In [10]:
dframe(mod_data[1]['EEG_filtered']).head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,-0.020422,-0.022875,-0.020752,-0.024290,-0.024856,-0.026978,-0.014055,-0.021743,-0.024573,-0.027733,...,-0.025186,-0.022356,-0.023960,-0.029572,-0.028629,-0.025752,-0.020234,-0.024573,-0.024054,-0.019007
1,-0.028404,-0.031205,-0.030831,-0.033074,-0.033981,-0.033931,-0.020491,-0.029673,-0.036051,-0.038065,...,-0.031740,-0.030597,-0.034277,-0.040461,-0.039797,-0.034346,-0.029374,-0.034872,-0.033981,-0.029271
2,-0.022534,-0.027174,-0.026464,-0.029447,-0.028894,-0.029548,-0.017482,-0.027437,-0.032539,-0.034971,...,-0.024285,-0.029289,-0.032084,-0.038319,-0.037874,-0.032493,-0.028467,-0.033443,-0.032392,-0.028682
3,-0.027857,-0.027150,-0.032594,-0.035113,-0.038213,-0.037143,-0.019260,-0.029577,-0.040505,-0.041637,...,-0.035856,-0.031609,-0.039735,-0.045916,-0.046540,-0.041085,-0.032311,-0.039749,-0.039492,-0.030589
4,-0.064138,-0.066045,-0.067683,-0.079740,-0.082914,-0.087771,-0.040368,-0.065460,-0.080927,-0.091818,...,-0.087361,-0.067454,-0.079225,-0.097632,-0.098159,-0.089933,-0.064292,-0.082052,-0.082849,-0.061782


### Convert samples x n_electrodes -> no_trials x n_electrodes x samples 
Remember to process the filtered EEG with shape of samples x electrodes

In [11]:
# Iterate overall all_data
for subj in mod_data.keys():
    print('Processing for subject', subj)
    mod_data[subj]['all_trials'], mod_data[subj]['y'], _ = process_s_data(data=mod_data[subj], eeg_key='EEG_filtered', start_t=0.5, end_t=3.5, fs=250)

Processing for subject 1
Processing for subject 2
Processing for subject 3
Processing for subject 4
Processing for subject 5
Processing for subject 6
Processing for subject 7
Processing for subject 8
Processing for subject 9


In [12]:
mod_data[2]['all_trials'].shape

(144, 22, 750)

In [108]:
dframe(mod_data[5]['all_trials'][6]).iloc[:10]

,0,1,2,3,4,5,6,7,8,9,...,740,741,742,743,744,745,746,747,748,749
0,2.831225,3.908294,4.449849,4.378324,3.742567,2.703605,1.307775,-0.408919,-2.290599,-4.026515,...,-3.934988,-3.078539,-2.011369,-0.861646,0.192806,0.994518,1.429399,1.528896,1.414543,1.317802
1,4.552508,5.900402,6.152449,5.226010,3.403669,1.273342,-0.632542,-1.959919,-2.534516,-2.362089,...,-4.288117,-4.128549,-3.374571,-2.117593,-0.630167,0.755886,1.699310,2.052092,1.845995,1.337277
2,4.040848,5.192079,5.402117,4.672942,3.285968,1.715717,0.279973,-0.873856,-1.699544,-2.192805,...,-5.330225,-4.911525,-4.056859,-2.824312,-1.390331,0.014043,1.130867,1.826411,2.081070,2.069634
3,3.429017,4.367521,4.607730,4.135970,3.124638,1.889428,0.611817,-0.651014,-1.836032,-2.805330,...,-5.934167,-5.142495,-3.934169,-2.513212,-1.105981,0.119408,1.018298,1.584724,1.865963,2.015373
4,3.608418,4.045117,4.080243,3.825804,3.409204,2.922179,2.212409,1.050827,-0.628231,-2.609484,...,-4.718487,-4.275966,-3.733652,-3.069747,-2.261631,-1.279024,-0.158012,1.047967,2.223742,3.281814
5,3.991446,4.108935,3.930405,3.574364,3.110047,2.547555,1.686041,0.297946,-1.609874,-3.700618,...,-3.648237,-3.251446,-2.710506,-2.094913,-1.448670,-0.720658,0.128474,1.159591,2.326735,3.512759
6,3.822723,5.666412,6.340582,5.781415,4.293528,2.477332,0.826080,-0.355372,-0.973503,-1.115745,...,-3.103053,-3.804356,-3.950902,-3.348587,-2.100468,-0.522067,0.955570,2.006688,2.452539,2.364979
7,3.740346,4.855241,5.011246,4.229082,2.809719,1.268153,-0.009607,-0.838442,-1.180222,-1.092079,...,-5.881179,-5.997063,-5.324623,-3.946722,-2.149172,-0.293058,1.224050,2.189421,2.533206,2.405074
8,3.354028,4.185194,4.205664,3.508617,2.399137,1.296841,0.408868,-0.249901,-0.745095,-1.133733,...,-5.957361,-5.987533,-5.491642,-4.454690,-3.001376,-1.341464,0.246939,1.559108,2.456476,2.974979
9,3.062213,3.871230,3.991827,3.514069,2.684576,1.821369,1.027980,0.235331,-0.629816,-1.515984,...,-6.550056,-6.024622,-5.098191,-3.890149,-2.519493,-1.067594,0.341394,1.628959,2.674813,3.423529


## Apply Data Alignment on raw EEG

In [14]:
from func.EA import apply_EA

In [15]:
mod_data[1].keys()

dict_keys(['s', 'etyp', 'epos', 'EEG_filtered', 'all_trials', 'y'])

In [16]:
# Iterate over all subjects to apply EA to all trials
# Define keys to be processed
process_key = ['all_trials']

for subj in mod_data.keys():
    print('Processing subject ', subj)
    mod_data[subj]['all_trials_EA']  = apply_EA(mod_data[subj], process_key)
    print('')

Processing subject  1
Found 1 key(s) in which EEG data is stored
Computing reference matrix RefEA
Add RefEA as a new key in data

Processing subject  2
Found 1 key(s) in which EEG data is stored
Computing reference matrix RefEA
Add RefEA as a new key in data

Processing subject  3
Found 1 key(s) in which EEG data is stored
Computing reference matrix RefEA
Add RefEA as a new key in data

Processing subject  4
Found 1 key(s) in which EEG data is stored
Computing reference matrix RefEA
Add RefEA as a new key in data

Processing subject  5
Found 1 key(s) in which EEG data is stored
Computing reference matrix RefEA
Add RefEA as a new key in data

Processing subject  6
Found 1 key(s) in which EEG data is stored
Computing reference matrix RefEA
Add RefEA as a new key in data

Processing subject  7
Found 1 key(s) in which EEG data is stored
Computing reference matrix RefEA
Add RefEA as a new key in data

Processing subject  8
Found 1 key(s) in which EEG data is stored
Computing reference matri

In [17]:
mod_data[1].keys()

dict_keys(['s', 'etyp', 'epos', 'EEG_filtered', 'all_trials', 'y', 'RefEA', 'R_inv', 'all_trials_EA'])

In [117]:
dframe(mod_data[7]['R_inv']).head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1.050787,-0.137931,-0.284588,-0.398178,-0.248363,-0.180263,-0.068785,0.112794,0.049804,0.161666,...,-0.075583,0.053906,-0.110657,0.012190,-0.006318,-0.044003,0.028455,-0.018040,0.016138,0.002165
1,-0.137931,1.275520,-0.654065,-0.031563,0.152953,-0.069335,-0.210779,-0.586293,0.093911,-0.038425,...,0.059156,0.163120,0.158174,-0.106424,0.001245,-0.000951,-0.037840,0.007077,-0.009058,0.005652
2,-0.284588,-0.654065,2.009974,-0.555685,-0.116620,0.157323,0.033919,0.075621,-0.913413,0.108439,...,-0.051272,0.135705,-0.040324,0.193394,-0.001307,-0.025685,-0.045824,0.017960,-0.004297,0.005540
3,-0.398178,-0.031563,-0.555685,1.857500,-0.404813,-0.078543,0.190657,-0.175220,0.122410,-0.622188,...,0.105236,-0.174931,0.396956,-0.033827,0.073182,0.061909,-0.041013,-0.030554,-0.044469,0.014605
4,-0.248363,0.152953,-0.116620,-0.404813,1.648440,-0.466228,-0.099422,0.077973,-0.124647,0.029821,...,0.040989,0.085390,-0.115529,0.122820,0.035246,0.036044,-0.001525,0.022960,0.006968,-0.013308
